- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb
- https://www.kaggle.com/code/ahmecse/sentiment-analysis-a-comprehensive-nlp-guide

In [2]:
!pip install openai


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import openai

from dotenv import load_dotenv
import os

load_dotenv()

username = os.getenv('USERNAME')
apiKey = os.getenv('OPENAI_API_KEY')

# Replace 'YOUR_OPENAI_API_KEY' with your actual API key
openai.api_key = apiKey

In [4]:
GPT_MODEL = "gpt-3.5-turbo-0613"
import requests

def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [5]:
from termcolor import colored

def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [23]:
functions = [
    {
        "name": "get_position_job",
        "description": "Get the job that is being recruited",
        "parameters": {
            "type": "object",
            "properties": {
                "position": {
                    "type": "string",
                    "description": "The language programing for developer",
                },
                "experience": {
                    "type": "string",
                    "description": "The experience for developer",
                },
            },
            "required": ["position", "experience"],
        },
    },
]

In [13]:
prompt = 'muốn xây dựng BCP là gì'
response = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    max_tokens=500,
    top_p=1.0,
    n=1,
    stop=None,
    temperature=0.3
)

print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "?\n\nBCP l\u00e0 vi\u1ebft t\u1eaft c\u1ee7a Business Continuity Plan, \u0111\u00e2y l\u00e0 m\u1ed9t k\u1ebf ho\u1ea1ch qu\u1ea3n l\u00fd r\u1ee7i ro \u0111\u1ec3 \u0111\u1ea3m b\u1ea3o s\u1ef1 li\u00ean t\u1ee5c c\u1ee7a ho\u1ea1t \u0111\u1ed9ng kinh doanh c\u1ee7a m\u1ed9t doanh nghi\u1ec7p. K\u1ebf ho\u1ea1ch n\u00e0y bao g\u1ed3m c\u00e1c quy tr\u00ecnh, quy \u0111\u1ecbnh v\u00e0 b\u1ea3n k\u1ebf ho\u1ea1ch \u0111\u1ec3 gi\u1ea3i quy\u1ebft c\u00e1c v\u1ea5n \u0111\u1ec1 v\u1ec1 an to\u00e0n v\u00e0 b\u1ea3o m\u1eadt, \u0111\u1ec3 \u0111\u1ea3m b\u1ea3o r\u1eb1ng doanh nghi\u1ec7p c\u00f3 th\u1ec3 ti\u1ebfp t\u1ee5c ho\u1ea1t \u0111\u1ed9ng m\u00e0 kh\u00f4ng b\u1ecb gi\u00e1n \u0111o\u1ea1n."
    }
  ],
  "created": 1690640746,
  "id": "cmpl-7hfB49YUlRbjXBQItslycMwOuyaec",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens"

In [56]:
%load_ext autoreload
%autoreload 2

import json

def execute_function_call(message):
    paramFn = json.loads(message["function_call"]["arguments"])
    results = globals()[message["function_call"]["name"]](paramFn)

    return results


messages = []
def establish_context(system_message, user_message, functions=None):
    messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": user_message})
    chat_response = chat_completion_request(messages, functions)
    assistant_message = chat_response.json()["choices"][0]["message"]
    messages.append(assistant_message)
    if assistant_message.get("function_call"):
        results = execute_function_call(assistant_message)
        messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

    pretty_print_conversation(messages)

def generate_message(user_message, functions=None):
    messages.append({"role": "user", "content": user_message})
    chat_response = chat_completion_request(messages, functions)
    assistant_message = chat_response.json()["choices"][0]["message"]
    messages.append(assistant_message)
    if assistant_message.get("function_call"):
        results = execute_function_call(assistant_message)
        messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

    pretty_print_conversation(messages)

functionsStock = [
    {
        "name": "get_stock_key",
        "description": "Lấy mã stock từ message",
        "parameters": {
            "type": "object",
            "properties": {
                "stock_key": {
                    "type": "string",
                    "description": "Các mã stock của các công ty ở Việt Nam",
                },
                "action": {
                    "type": "string",
                    "enum": ["so sánh", "báo cáo tài chính", "thông tin cơ bản"],
                    "description": "Lấy ra hành động mà người dùng mong muốn thực hiện",
                },
            },
            "required": ["location", "format"],
        },
    },
]

from vnstock import *

def trim_words(sentence, num_word=200):
    # Split the sentence into words
    words = sentence.split()
    # Trim the sentence to 5 words
    trimmed_sentence = " ".join(words[:num_word])
    return trimmed_sentence

def compare_stock(stock_key):
    datals = stock_ls_analysis(stock_key, lang='vi')
    dataCompare = datals.to_string()
    # print('compare_stock', stock_key, dataCompare)
    prompt = "So sánh các mã " + stock_key + " dựa vào dữ liệu sau: \n" + dataCompare + ".\n"
    prompt = trim_words(prompt, 200)

    print('compare_stock prompt', prompt)
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        max_tokens=1000,
        top_p=1.0,
        n=1,
        stop=None,
        temperature=0.3
    )

    res = response["choices"][0]["text"]
    print('compare_stock:res:', res)
    return res

from datetime import datetime, timedelta
from importlib import reload
import algorithm.rnn as stockRnn

reload(stockRnn)

def get_last_days(num_days):
        # Get the current date
    current_date = datetime.now()

    # Calculate the date from 1 year ago
    days_ago = current_date - timedelta(days=num_days)

    # Create a list to store the dates for the last 1 year
    date_list = [days_ago + timedelta(days=i) for i in range(num_days)]

    # Format the dates as "%Y-%m-%d" and store them in a new list
    formatted_dates = [date.strftime("%Y-%m-%d") for date in date_list]

    return formatted_dates

def predict_stock(stock_key):
    # Get the current date
    dates_train = get_last_days(365)
    stock_codes = stock_key.split(",")
    for code in stock_codes:
        df_his = stock_historical_data(code, dates_train[0], dates_train[len(dates_train) - 1], "1D", 'stock')
        data_train = df_his.iloc[:, 1:2].values

        
        # data_test = [data_train[:10]]
        # rnn = stockRnn.TrainRnn()
        # rnn.fit(data_train)
        # predict_price = rnn.predict(data_test)
        # print('predict_price', predict_price)


def get_stock_key(params):
    stock_key, action = params["stock_key"], params["action"]

    switcher = {
        "so sánh": compare_stock,
        "báo cáo tài chính": None,
        "thông tin cơ bản": None,
        "dự đoán": predict_stock
    }
    return switcher.get(action)(stock_key)

get_stock_key({"stock_key": "TPB", "action": "dự đoán"})


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
code TPB 2022-07-30 2023-07-29
df_his (250, 1) [[17540]
 [17670]
 [17800]
 [18450]
 [18710]
 [18610]
 [18740]
 [18580]
 [18550]
 [18550]
 [18740]
 [18770]
 [18680]
 [18580]
 [18640]
 [18290]
 [17800]
 [18130]
 [18420]
 [18680]
 [18130]
 [18130]
 [18060]
 [18450]
 [18230]
 [18000]
 [17710]
 [17670]
 [17480]
 [17540]
 [17160]
 [17260]
 [17160]
 [17030]
 [16450]
 [16190]
 [16190]
 [16420]
 [15990]
 [16059]
 [15700]
 [15990]
 [15860]
 [16250]
 [15480]
 [15540]
 [15480]
 [14920]
 [13460]
 [12530]
 [11850]
 [13270]
 [13560]
 [13010]
 [13400]
 [13400]
 [13140]
 [13330]
 [13010]
 [12750]
 [13080]
 [12920]
 [13730]
 [13590]
 [13590]
 [14120]
 [13820]
 [13590]
 [13890]
 [12750]
 [13400]
 [12980]
 [13270]
 [12620]
 [12560]
 [12140]
 [13790]
 [13330]
 [13400]
 [13270]
 [13180]
 [13010]
 [13080]
 [13270]
 [13790]
 [14050]
 [14470]
 [13920]
 [15020]
 [14890]
 [13920]
 [13820]
 [14380]
 [14730]
 [14630]
 [15150]
 

In [36]:
establish_context("Trả lời câu hỏi của người dùng về tình trạng các mã stock ?",
                  "So sánh 2 cổ phiếu TPB, VPB ở thời điểm hiện tại", functionsStock)

compare_stock prompt So sánh các mã TPB, VPB dựa vào dữ liệu sau: Mã CP TPB VPB Vốn hóa (tỷ) 41171 148362 Giá 18700 22100 Số phiên tăng/giảm liên tiếp 1 1 P/E 6.9 12.8 PEG 0.8 -0.4 P/B 1.3 1.4 Cổ tức 0.0 0.0 ROE 0.198 0.12 ROA 0.018 0.017 interestMargin 0.037 0.062 Tỉ lệ nợ xấu 0.022 0.074 Nợ/Vốn CSH 10.1 5.9 LNST 5 năm 0.454 0.23 Doanh thu 5 năm 0.34 0.182 LNST quý gần nhất -0.085 0.207 Doanh thu quý gần nhất 0.071 0.034 LNST năm tới 0.074 0.006 Doanh thu năm tới 0.124 0.063 RSI 55.7 73.9 RS 40.0 67.0.
compare_stock:res: 

Mã CP TPB có giá trần cao hơn so với VPB, với P/E là 6.9 so với 12.8 của VPB. VPB có PEG là -0.4, trong khi TPB có PEG là 0.8. Cổ tức của cả hai là 0.0. TPB có ROE cao hơn so với VPB (0.198 so với 0.12). ROA của TPB là 0.018, cao hơn so với 0.017 của VPB. TPB cũng có interestMargin cao hơn so với VPB (0.037 so với 0.062). Tỉ lệ nợ xấu của TPB là 0.022, thấp hơn so với 0.074 của VPB. Nợ/Vốn CSH của TPB là 10.1, cao hơn so với 5.9 của VPB. LNST 5 năm của TPB là 0.454,

In [ ]:
generate_message("Có cần kinh nghiệm về vuejs không ?", functions)